# TF-IDF値の算出結果比較



## (1) 指定BotのTFIDF値を、全Botの学習セットを使用して算出

ベクトライズにはすべてのBotの学習セットを使用しますが、ターゲット「test_toyotsu_human_conversation.csv」に対応するTFIDF値のベクトルだけを取得するようにします。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
    'test_daikin_conversation.csv',
    'test_ptna_conversation.csv',
    'test_toyotsu_human_conversation.csv'
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_toyotsu_human_conversation.csv]


In [3]:
import numpy as np
import pandas as pd


def get_dataset_names_array(dataset_name, count):
    '''
        データセット名称のリストを生成
    '''
    dataset_name_list = [dataset_name] * count
    dataset_names = np.array(dataset_name_list)
    return dataset_names


'''
    学習セットの全CSVファイルから、
    質問文の配列を生成
'''
questions_all = np.empty(0)
answer_ids_all = np.empty(0)
dataset_names_all = np.empty(0)

for csv_file_path in temp_path:
    '''
        TrainingMessageFromCsv と同様の処理

        CSVファイル--->pandasデータフレームの生成
        (TrainingMessageFromCsv#__build_learning_training_messages)
    '''
    learning_training_messages = pd.read_csv(csv_file_path, encoding='utf-8')
    questions = np.array(learning_training_messages['question'])
    answer_ids = np.array(learning_training_messages['answer_id'])
    dataset_names = get_dataset_names_array(os.path.basename(csv_file_path), learning_training_messages.shape[0])

    questions_all = np.concatenate((questions_all, questions))
    answer_ids_all = np.concatenate((answer_ids_all, answer_ids))
    dataset_names_all = np.concatenate((dataset_names_all, dataset_names))

In [4]:
'''
    TextArray#__init__ と同様の処理
'''
from learning.core.nlang import Nlang
from sklearn.feature_extraction.text import TfidfVectorizer

sentences = np.array(questions_all)
separated_sentences = Nlang.batch_split(sentences)
np.size(separated_sentences)

31180

In [5]:
'''
    TextArray#to_vec と同様の処理
'''
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
vectorizer.fit(separated_sentences)
feature_vectors = vectorizer.transform(separated_sentences)

In [6]:
feature_vectors

<31180x3162 sparse matrix of type '<class 'numpy.float64'>'
	with 243448 stored elements in Compressed Sparse Row format>

In [7]:
'''
    指定Botに対応するデータのインデックスを取得
'''
target_data_set_name = 'test_toyotsu_human_conversation.csv'
target_indices = np.where(dataset_names_all==target_data_set_name)
np.size(target_indices)

5064

In [8]:
target_indices

(array([26116, 26117, 26118, ..., 31177, 31178, 31179]),)

In [9]:
'''
    指定Botに対応するTFIDF値だけを抽出する
    --->これが指定Botの学習＋予測に使用されるベクトルになります
'''
target_feature_vectors = feature_vectors[target_indices]
target_feature_vectors

<5064x3162 sparse matrix of type '<class 'numpy.float64'>'
	with 49382 stored elements in Compressed Sparse Row format>

### (1-1) 指定Botにおける頻出単語を確認

In [10]:
def get_words_with_frequency(vectorizer, feature_vectors):
    bools = (feature_vectors > 0)
    sums = np.sum(bools, axis=0)
    frequency_array = np.array(sums)[0]
    
    words_with_frequency = []
    for item in vectorizer.vocabulary_.items():
        k, v = item
        freq = frequency_array[v] # 対象Botに出現しない単語の場合、値は0
        if freq > 0:
            words_with_frequency.append((k, v, freq))

    return words_with_frequency

In [11]:
'''
    ボキャブラリの単語について、
    インデックスおよび出現数を取得
'''
words_with_frequency = get_words_with_frequency(vectorizer, target_feature_vectors)
len(words_with_frequency)

1182

In [12]:
'''
    出現回数の降順に並べ替えます
'''
sorted_words_list_target = sorted(words_with_frequency, key=lambda x: x[2], reverse=True)
sorted_words_list_target[:20] # 上位２０件を表示

[('する', 159, 2722),
 ('出張', 1216, 800),
 ('保険', 1109, 770),
 ('場合', 1455, 766),
 ('海外', 2093, 744),
 ('退職', 2642, 744),
 ('申請', 2193, 600),
 ('なる', 232, 564),
 ('科目', 2270, 522),
 ('勘定', 1274, 512),
 ('書', 1929, 504),
 ('れる', 322, 498),
 ('精算', 2319, 482),
 ('健康', 1129, 476),
 ('教える', 1846, 456),
 ('どう', 212, 454),
 ('できる', 200, 438),
 ('ない', 221, 432),
 ('いい', 14, 424),
 ('金', 2724, 384)]

### (1-2) 全Botにおける頻出単語を確認

In [13]:
words_with_frequency_all = get_words_with_frequency(vectorizer, feature_vectors)
len(words_with_frequency_all)

3162

In [14]:
'''
    出現回数の降順に並べ替えます
'''
sorted_words_list_all = sorted(words_with_frequency_all, key=lambda x: x[2], reverse=True)
sorted_words_list_all[:20] # 上位２０件を表示

[('する', 159, 15282),
 ('メール', 819, 4790),
 ('ない', 221, 4431),
 ('れる', 322, 3811),
 ('できる', 200, 3734),
 ('教える', 1846, 2828),
 ('２０１０', 2855, 2700),
 ('ｏｕｔｌｏｏｋ', 3071, 2515),
 ('暗号', 1923, 2252),
 ('どう', 212, 1958),
 ('設定', 2509, 1940),
 ('表示', 2436, 1935),
 ('方法', 1876, 1930),
 ('なる', 232, 1776),
 ('インストール', 375, 1760),
 ('ｅ', 2959, 1721),
 ('管理', 2312, 1692),
 ('いい', 14, 1643),
 ('知る', 2239, 1575),
 ('利用', 1252, 1528)]

### (1-3) 調査で使用する最頻出単語を決める

In [15]:
'''
    ここで調査する最頻出単語は、
    全Bot、指定Bot で共通して
    頻出している５件の単語に限定してみます。
'''
most_freq_words = ['する', 'れる', 'どう', 'できる', 'ない']

## (2) 単独BotにおけるTFIDF値を算出

プロダクションコードと同様の処理で、単独Botの学習セットを使用してTFIDF値を算出します。

ターゲットは「test_toyotsu_human_conversation.csv」です。

In [16]:
def get_tfidf_for_one_target(csv_file_path):
    '''
        TrainingMessageFromCsv と同様の処理

        CSVファイル--->pandasデータフレームの生成
        (TrainingMessageFromCsv#__build_learning_training_messages)
    '''
    learning_training_messages = pd.read_csv(csv_file_path, encoding='utf-8')
    questions = np.array(learning_training_messages['question'])
    answer_ids = np.array(learning_training_messages['answer_id'])

    '''
        TextArray#__init__ と同様の処理
    '''
    sentences = np.array(questions)
    separated_sentences = Nlang.batch_split(sentences)
    np.size(separated_sentences)

    '''
        TextArray#to_vec と同様の処理
    '''
    vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
    vectorizer.fit(separated_sentences)
    feature_vectors = vectorizer.transform(separated_sentences)
    
    return questions, vectorizer, feature_vectors

In [17]:
'''
    ベクトライザー＆featureベクトルを生成
'''
csv_file_path = temp_path[3]
questions, vectorizer_toyotsu, feature_vectors_toyotsu = get_tfidf_for_one_target(csv_file_path)

In [18]:
'''
    ボキャブラリの単語について、
    インデックスおよび出現数を取得
'''
words_with_frequency_toyotsu = get_words_with_frequency(vectorizer_toyotsu, feature_vectors_toyotsu)
len(words_with_frequency_toyotsu)

1182

## (3) TFIDF値を比較

In [19]:
'''
    再頻出単語のインデックスを取得しておく
'''
index_target = []
for word in most_freq_words:
    for w, i, _ in words_with_frequency:
        if w == word:
            index_target.append(i)

index_target

[159, 322, 212, 200, 221]

In [20]:
'''
    再頻出単語のインデックスを取得しておく
'''
index_toyotsu = []
for word in most_freq_words:
    for w, i, _ in words_with_frequency_toyotsu:
        if w == word:
            index_toyotsu.append(i)

index_toyotsu

[51, 121, 71, 65, 80]

### (3-1) TF-IDF値の変化を２０件ほど確認

再頻出単語 'する', 'れる', 'どう', 'できる', 'ない' に対するTFIDF値が、概ね下がったことは確認できました。

In [21]:
display_cnt = 0
for index in range(feature_vectors_toyotsu.shape[0]):
    data_toyotsu = feature_vectors_toyotsu.getrow(index).toarray()[0]
    data_target = target_feature_vectors.getrow(index).toarray()[0]

    if data_toyotsu[index_toyotsu[0]] == 0 and \
        data_toyotsu[index_toyotsu[1]] == 0 and \
        data_toyotsu[index_toyotsu[2]] == 0 and \
        data_toyotsu[index_toyotsu[3]] == 0 and \
        data_toyotsu[index_toyotsu[4]] == 0:
        continue
    
    display_cnt += 1
    print('index=%-2d [%s]%0.3f->%0.3f [%s]%0.3f->%0.3f [%s]%0.3f->%0.3f [%s]%0.3f->%0.3f [%s]%0.3f->%0.3f' % (
        index,
        most_freq_words[0],
        data_toyotsu[index_toyotsu[0]],
        data_target[index_target[0]],
        most_freq_words[1],
        data_toyotsu[index_toyotsu[1]],
        data_target[index_target[1]],
        most_freq_words[2],
        data_toyotsu[index_toyotsu[2]],
        data_target[index_target[2]],
        most_freq_words[3],
        data_toyotsu[index_toyotsu[3]],
        data_target[index_target[3]],
        most_freq_words[4],
        data_toyotsu[index_toyotsu[4]],
        data_target[index_target[4]],
    ))
    
    if display_cnt == 20:
        break

index=0  [する]0.159->0.132 [れる]0.000->0.000 [どう]0.000->0.000 [できる]0.000->0.000 [ない]0.000->0.000
index=1  [する]0.055->0.046 [れる]0.000->0.000 [どう]0.000->0.000 [できる]0.000->0.000 [ない]0.000->0.000
index=2  [する]0.062->0.049 [れる]0.000->0.000 [どう]0.000->0.000 [できる]0.000->0.000 [ない]0.000->0.000
index=6  [する]0.000->0.000 [れる]0.000->0.000 [どう]0.000->0.000 [できる]0.198->0.138 [ない]0.000->0.000
index=7  [する]0.098->0.080 [れる]0.000->0.000 [どう]0.000->0.000 [できる]0.000->0.000 [ない]0.000->0.000
index=9  [する]0.116->0.093 [れる]0.000->0.000 [どう]0.122->0.102 [できる]0.000->0.000 [ない]0.000->0.000
index=10 [する]0.104->0.091 [れる]0.000->0.000 [どう]0.219->0.201 [できる]0.000->0.000 [ない]0.222->0.157
index=11 [する]0.121->0.097 [れる]0.000->0.000 [どう]0.000->0.000 [できる]0.000->0.000 [ない]0.000->0.000
index=12 [する]0.000->0.000 [れる]0.000->0.000 [どう]0.184->0.185 [できる]0.000->0.000 [ない]0.000->0.000
index=13 [する]0.093->0.071 [れる]0.000->0.000 [どう]0.000->0.000 [できる]0.000->0.000 [ない]0.000->0.000
index=14 [する]0.000->0.000 [れる]0.000->0.000 [どう]0.0

### (3-2) 任意の質問文について、TF-IDF値の変化を確認

頻出単語の重みが低下し、その他の単語の重みが増していることが確認できます。

In [22]:
def get_item_from_vocabulary(vocabulary, index):
    '''
        vocabulary から指定インデックスの単語を参照
    '''
    for k, v in vocabulary.items():
        if v == index:
            return k

    return None

def show_tfidf_values(question_index):
    a = feature_vectors_toyotsu.getrow(question_index)
    voc_idx_toyotsu = np.nonzero(a)[1]
    data_toyotsu = np.array(a[np.nonzero(a)])[0]
    b = target_feature_vectors.getrow(question_index)
    data_target = np.array(b[np.nonzero(b)])[0]

    print('question=%s' % questions[question_index])

    for i, v in enumerate(data_toyotsu):
        if v > 0: # TF-IDF値が0でないfeature
            print('feature #%-2d %0.6f --> %0.6f (%s)' % (
                i,
                data_toyotsu[i],
                data_target[i],
                get_item_from_vocabulary(vectorizer_toyotsu.vocabulary_, voc_idx_toyotsu[i]),
            ))

In [23]:
show_tfidf_values(10)

question=「転勤者引越対応」引っ越し先が決まっていないが、どうしたらいいか。
feature #0  0.223013 --> 0.210334 (いい)
feature #1  0.103915 --> 0.091389 (する)
feature #2  0.218639 --> 0.200982 (どう)
feature #3  0.221817 --> 0.157428 (ない)
feature #4  0.255266 --> 0.254752 (先)
feature #5  0.334237 --> 0.259754 (対応)
feature #6  0.422277 --> 0.448290 (引っ越し)
feature #7  0.356499 --> 0.393562 (引越)
feature #8  0.386878 --> 0.386063 (決まる)
feature #9  0.233124 --> 0.252416 (者)
feature #10 0.395164 --> 0.425731 (転勤)


In [24]:
show_tfidf_values(853)

question=「VISA」VISA取得のための代金はどの勘定科目なのか？
feature #0  0.428232 --> 0.401426 (代金)
feature #1  0.223223 --> 0.263109 (勘定)
feature #2  0.323515 --> 0.299934 (取得)
feature #3  0.221913 --> 0.262114 (科目)
feature #4  0.782868 --> 0.781643 (ｖｉｓａ)


In [25]:
show_tfidf_values(975)

question=「勘定科目」出張していないが、現地でTMCとの交際費をTTC負担にしてほしいと 現地から依頼があったのですがその時のその処理について教えてもらえますか？
feature #0  0.170939 --> 0.137977 (する)
feature #1  0.182443 --> 0.118841 (ない)
feature #2  0.262441 --> 0.207446 (ほしい)
feature #3  0.318206 --> 0.316178 (交際)
feature #4  0.309395 --> 0.286473 (依頼)
feature #5  0.206003 --> 0.217873 (処理)
feature #6  0.150003 --> 0.182998 (出張)
feature #7  0.173502 --> 0.205681 (勘定)
feature #8  0.179598 --> 0.136920 (教える)
feature #9  0.473246 --> 0.507760 (現地)
feature #10 0.172484 --> 0.204903 (科目)
feature #11 0.195663 --> 0.219832 (負担)
feature #12 0.191453 --> 0.207775 (費)
feature #13 0.386728 --> 0.368502 (ｔｍｃ)
feature #14 0.257163 --> 0.269565 (ｔｔｃ)
